# Setup

In [1]:
%matplotlib ipympl
import matplotlib, matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Number of custom components

In [2]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
        {
            "Unique custom components": lambda app: app.components.filter(lambda component: component.type == "Custom")
            .map(lambda component: component[".tagName"])
            .filter(lambda tagName: tagName != None)
            .unique()
            .length
        }
    ).order_by(lambda app: (app["Unique custom components"],), reverse=True).table()

Env,Org,App,Unique custom components
tt02,dibk,fa-v3,10
tt02,dibk,ig-v3,10
tt02,dibk,mb-v3,10
tt02,dibk,innsending-planforslag,6
tt02,dat,byggesak-samtykke-v3,6
tt02,digdir,dan-data-app,1


Count: 6


## Custom components per app

In [3]:
with Apps.init() as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
        {
            "Custom component tags": lambda app: app.components.filter(lambda component: component.type == "Custom")
            .map(lambda component: component[".tagName"])
            .unique()
            .sort()
        }
    ).order_by(lambda app: (app.org, app.app)).table()

Env,Org,App,Custom component tags
tt02,dat,byggesak-samtykke-v3,"[custom-field-adresse, custom-field-data, custom-field-kommunens-saksnummer, custom-field-part-navn, custom-field-prosjekt, custom-field-telefonnummer]"
tt02,dibk,fa-v3,"[custom-field-adresse, custom-field-boolean-text, custom-field-data, custom-field-kommunens-saksnummer, custom-field-prosjekt, custom-grouplist-utfall-svar-type, custom-header-text, custom-list-data, custom-subheader-text, custom-table-data]"
tt02,dibk,ig-v3,"[custom-field-adresse, custom-field-boolean-text, custom-field-data, custom-field-kommunens-saksnummer, custom-field-prosjekt, custom-grouplist-utfall-svar-type, custom-header-text, custom-list-data, custom-subheader-text, custom-table-data]"
tt02,dibk,innsending-planforslag,"[custom-field-adresse, custom-field-data, custom-field-kommunens-saksnummer, custom-header-text, custom-paragraph-text, custom-table-data]"
tt02,dibk,mb-v3,"[custom-field-adresse, custom-field-boolean-text, custom-field-data, custom-field-kommunens-saksnummer, custom-field-prosjekt, custom-grouplist-utfall-svar-type, custom-header-text, custom-list-data, custom-subheader-text, custom-table-data]"
tt02,digdir,dan-data-app,[my-component]


Count: 6


## Apps using Subform in prod

In [4]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod" and app.components.some(lambda component: component.type == "Subform")
    ).table()

Count: 0


## RuleConfiguration conditional rendering rules

In [5]:
with Apps.init() as apps:
    apps.where(lambda app: app.env == "prod").select(
        {
            "Frontend version": lambda app: app.frontend_version,
            "Rule config": lambda app: app.rule_configurations.flat_map(
                lambda rule_configuration: rule_configuration[".data.conditionalRendering.[].selectedFunction", :]
                .map(lambda func_name: rule_configuration.layout_set.rule_handler.conditional_rule(func_name))
                .filter(lambda func: func != None),
            ),
        }
    ).where(lambda app: app["Rule config"].length > 0).table()

Env Org App Frontend version Rule config 
 
 
 prod digdir be-om-api-nokkel 4 [isNotEmpty: (obj) => {
 if (!obj.value || obj.value === "") return false;
 return true;
 }, isWebApp: (obj) => {
 if (!obj.value) return false;
 return obj.value === "nettleserapplikasjon";
 }, isNotEmpty: (obj) => {
 if (!obj.value || obj.value === "") return false;
 return true;
 }] 
 prod dibk uttalelse-hoffe 3 [sjekkUttalelseFristDato: function (obj) {
 var date = Date.parse(obj.value);
 return date > Date.now();
 }, sjekkUttalelseFristDato: function (obj) {
 var date = Date.parse(obj.value);
 return date > Date.now();
 }] 
 prod dibk uttalelse-varselplanoppstart 3 [fristForInnspillGyldig: function (obj) {
 var fristForInnspillAsMillis = Date.parse(obj.value);
 var fristForInnspillAsDate = new Date(fristForInnspillAsMillis).setHours(0, 0, 0, 0);
 var todayAsDate = new Date().setHours(0,0,0,0);
 return fristForInnspillAsDate >= todayAsDate;
 }, fristForInnspillGyldig: function (obj) {
 var fristForInnspillAsMillis = Date.parse(obj.value);
 var fristForInnspillAsDate = new Date(fristForInnspillAsMillis).setHours(0, 0, 0, 0);
 var todayAsDate = new Date().setHours(0,0,0,0);
 return fristForInnspillAsDate >= todayAsDate;
 }] 
 prod brg anonym-oppstartsveilederen 3 [isOverskudd: (obj) => {
 return obj.resultat >= 0;
 }, isUnderskudd: (obj) => {
 return obj.resultat < 0;
 }, isUnderskudd: (obj) => {
 return obj.resultat < 0;
 }, isBehov: (obj) => {
 return (obj.value && obj.value === "behov");
 }, isArbeidsledig: (obj) => {
 return (obj.value && obj.value === "arbeidsledig");
 }, isOrgOppdrag: (obj) => {
 return (obj.value && obj.value === "orgOppdrag");
 }, isHobby: (obj) => {
 return (obj.value && obj.value === "hobby");
 }, isMesser: (obj) => {
 return (obj.value && obj.value === "messer");
 }, isArbeidsgiver: (obj) => {
 return (obj.value && obj.value === "informasjon");
 }, isArbeidsgiverVidere: (obj) => {
 return (obj.value && obj.value === "videre");
 }, isKjennerMarkedKundeOrTrorKjennerKundeOrKjennerIkkeKunde: (obj) => {
 return (obj.value == "kjennerMarkedKunde" || obj.value == "trorKjennerKunde" || obj.value == "kjennerIkkeKunde");
 }, iskonkurrenteneOversikt: (obj) => {
 return (obj.value && obj.value === "kjennerKonkurent");
 }, isvissOversiktKonkurrentene: (obj) => {
 return (obj.value && obj.value === "trorKjennerKonkurent");
 }, isikkeOversiktKonkurrentene: (obj) => {
 return (obj.value && obj.value === "kjennerIkkeKonkurent");
 }, isMindreVarierendeJa: (obj) => {
 return (obj.value && obj.value === "jaHarBuffer");
 }, isMindreVarierendeNei: (obj) => {
 return (obj.value && obj.value === "neiIngenMidler");
 }, isMindreVarierendeRelevant: (obj) => {
 return (obj.value && obj.value === "ikkeRelevant");
 }, isSkattJa: (obj) => {
 return (obj.value && obj.value === "ja");
 }, isSkattNei: (obj) => {
 return (obj.value && obj.value === "nei");
 }, isBudsjettOk: (obj) => {
 return (obj.value && obj.value === "kontroll");
 }, isBudsjettNei: (obj) => {
 return (obj.value && obj.value === "Nei/delvis");
 }, isJaMvaOrJaMvaHvordanOrNeiMva: (obj) => {
 return (obj.value == "jaMva" || obj.value == "jaMvaHvordan" || obj.value == "neiMva");
 }, isRegnskapJa: (obj) => {
 return (obj.value && obj.value === "ja");
 }, isRegnskapNei: (obj) => {
 return (obj.value && obj.value === "nei");
 }, isRegnskapRegnskapsfører: (obj) => {
 return (obj.value && obj.value === "neiRegnskapsfører");
 }, isRettigheterJa: (obj) => {
 return (obj.value && obj.value === "ja");
 }, isRettigheterNei: (obj) => {
 return (obj.value && obj.value === "nei");
 }, isAnsvarJa: (obj) => {
 return (obj.value && obj.value === "ja");
 }, isAnsvarNei: (obj) => {
 return (obj.value && obj.value === "nei");
 }, isdiskuterJa: (obj) => {
 return (obj.value && obj.value === "ja");
 }, isdiskuterLitt: (obj) => {
 return (obj.value && obj.value === "litt");
 }, isdiskuterNei: (obj) => {
 return (obj.value && obj.value === "nei");
 }, harVesentligeUtgifterOgEgenkapitalUnder15ProsentOgTrengerLaanE

Count: 58
